# Vertex AI Search - Search App engine 구성 및 관리
이 Notebook은 Vertex AI Search를 관리하기 위한 다양한 API활용에 대해서 설명합니다.
이 예제를 통해서 프로그래밍 방식으로 Vertex AI Search를 관리할 수 있으며, 필요에 따라서 타 시스템과의 연동을 통해서 CI/CD를 연계 할 수도 있습니다. 

여기서 이야기하는 API는 아래 URL을 기준으로 작성되었습니다. 
*  https://cloud.google.com/python/docs/reference/discoveryengine/0.11.4

In [32]:
%pip install --upgrade --quiet google-cloud-discoveryengine

Note: you may need to restart the kernel to use updated packages.


### GCP 기본 환경 구성
이 예제에서는 Service account를 사용해서 인증을 합니다.
다른 방법으로 Oauth를 사용해서 인증하는 방법도 있습니다.

In [33]:
# Project constant setting.
PROJECT_ID="ai-hangsik"
REGION="global"

# Create a creadential to authenticate an access to the GCP.
from google.oauth2 import service_account
import google.oauth2.credentials

# Location of service account. Use the service account having an IAM including Discovery engine access.
SVC_ACCOUNT_FILE = "/home/admin_/keys/ai-hangsik-71898c80c9a5.json"

credentials = service_account.Credentials.from_service_account_file(
    SVC_ACCOUNT_FILE, 
    scopes=['https://www.googleapis.com/auth/cloud-platform']
)


### Search App Engine 생성

Search app을 생성하기 위해서는 Data store 가 이미 생성되어 이어야 합니다. 이미 만들어진 data store를 engine 에 attach 하는 형태로 만들어집니다.  
engine을 만들때 SolutionType.SOLUTION_TYPE_SEARCH 을 변경해서 Search 스타일을 구성할 수 있습니다.

In [34]:
import argparse
from google.cloud.discoveryengine_v1 import (
    CreateDataStoreRequest,
    CreateEngineRequest,
    DataStore,
    DataStoreServiceClient,
    SolutionType,
    Engine,
    EngineServiceClient,
)

def create_engine(project_id:str, 
                  region:str,
                  engine_id:str, 
                  data_store_id: str):

    # API : https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1alpha.services.engine_service.EngineServiceClient
    engine_client = EngineServiceClient(credentials= credentials)

    parent = f"projects/{project_id}/locations/{region}"

    engine = Engine(
        # API : https://cloud.google.com/python/docs/reference/discoveryengine/0.11.4/google.cloud.discoveryengine_v1alpha.types.Engine
        display_name=engine_id,
        data_store_ids=[data_store_id],
        solution_type = SolutionType.SOLUTION_TYPE_SEARCH,
    )

    request = CreateEngineRequest(
        # API : https://cloud.google.com/python/docs/reference/discoveryengine/0.11.4/google.cloud.discoveryengine_v1alpha.types.CreateEngineRequest
        parent=parent, 
        engine=engine, 
        engine_id=engine_id)
        
    print(f"Request: {request}")

    operation = engine_client.create_engine(
        # API : https://cloud.google.com/python/docs/reference/discoveryengine/0.11.4/google.cloud.discoveryengine_v1alpha.services.engine_service.EngineServiceClient#google_cloud_discoveryengine_v1alpha_services_engine_service_EngineServiceClient_create_engine
        request=request)

    print(f"Operation: {operation}")

    response = operation.result()
    print(f"Engine: {response}")



In [35]:
ENGINE_ID = "data_engine_001"
DATA_STORE_ID = "data_store_001"

create_engine(
    PROJECT_ID,
    REGION,
    ENGINE_ID, 
    DATA_STORE_ID )

Request: parent: "projects/ai-hangsik/locations/global"
engine {
  display_name: "data_engine_001"
  data_store_ids: "data_store_001"
  solution_type: SOLUTION_TYPE_SEARCH
}
engine_id: "data_engine_001"

Operation: <google.api_core.operation.Operation object at 0x78363c16ba90>
Engine: search_engine_config {
  search_tier: SEARCH_TIER_STANDARD
}
name: "projects/721521243942/locations/global/collections/default_collection/engines/data_engine_001"
display_name: "data_engine_001"
data_store_ids: "data_store_001"
solution_type: SOLUTION_TYPE_SEARCH



In [36]:
from typing import Any

def list_engines(  project_id: str, 
                   location: str, ) -> Any:

    from typing import Optional
    from google.api_core.client_options import ClientOptions
    from google.cloud import discoveryengine

    parent = f"projects/{project_id}/locations/{location}"

    #  For more information, refer to:
    # https://cloud.google.com/generative-ai-app-builder/docs/locations#specify_a_multi-region_for_your_data_store
    client_options = (
        ClientOptions(api_endpoint=f"{location}-discoveryengine.googleapis.com")
        if location != "global"
        else None
    )

    # Create a client
    # API : https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1alpha.services.engine_service.EngineServiceClient#google_cloud_discoveryengine_v1alpha_services_engine_service_EngineServiceClient
    client = discoveryengine.EngineServiceClient(client_options=client_options, 
                                                    credentials=credentials)

    response = client.list_engines(parent=parent)

    print(f"Engine lists")
    for result in response:
        print(result)

In [ ]:
list_engines(PROJECT_ID,REGION)

### More operation regarding engine

* Delete engine
    * https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1alpha.services.engine_service.EngineServiceClient#google_cloud_discoveryengine_v1alpha_services_engine_service_EngineServiceClient_delete_engine
* get engine
    * https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1alpha.services.engine_service.EngineServiceClient#google_cloud_discoveryengine_v1alpha_services_engine_service_EngineServiceClient_get_engine
* pause_engine
    * https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1alpha.services.engine_service.EngineServiceClient#google_cloud_discoveryengine_v1alpha_services_engine_service_EngineServiceClient_pause_engine
* resume_engine
    * https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1alpha.services.engine_service.EngineServiceClient#google_cloud_discoveryengine_v1alpha_services_engine_service_EngineServiceClient_resume_engine

